In [136]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import PIL

In [137]:
#!curl -O https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar

In [138]:
#!curl -O https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar

In [139]:
SEED=1

In [140]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [141]:
torch.cuda.device_count()

1

In [142]:
START_EPOCH = 0

In [143]:
ARCH = 'resnet18'
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
TRAINDIR="/w251-hw5/train"
VALDIR="/w251-hw5/val"

In [144]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [145]:
GPU = 0

In [146]:
torch.cuda.set_device(GPU)

In [147]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [148]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [149]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [150]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [151]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [152]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [153]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [154]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [155]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [156]:
IMG_SIZE = 224

In [157]:
NUM_CLASSES = 1000

In [158]:
model = models.__dict__[ARCH]()

In [159]:
inf = model.fc.in_features

In [160]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [161]:
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [162]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [163]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [164]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [127]:
# transform_train = transforms.Compose([
#     transforms.RandomChoice([transforms.Resize(256), transforms.Resize(480)]),
#     transforms.RandomCrop(IMG_SIZE, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
# ])

In [165]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [166]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [167]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [168]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [169]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [170]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [171]:
best_acc1 = 0

In [172]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][    0/10010]	Time  1.489 ( 1.489)	Data  1.228 ( 1.228)	Loss 7.0214e+00 (7.0214e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][   50/10010]	Time  0.587 ( 0.601)	Data  0.000 ( 0.035)	Loss 6.9409e+00 (7.1671e+00)	Acc@1   0.00 (  0.14)	Acc@5   0.00 (  0.69)
Epoch: [0][  100/10010]	Time  0.593 ( 0.596)	Data  0.000 ( 0.022)	Loss 6.8117e+00 (7.0271e+00)	Acc@1   0.00 (  0.22)	Acc@5   2.34 (  1.01)
Epoch: [0][  150/10010]	Time  0.589 ( 0.598)	Data  0.000 ( 0.017)	Loss 6.7491e+00 (6.9549e+00)	Acc@1   0.00 (  0.28)	Acc@5   1.56 (  1.25)
Epoch: [0][  200/10010]	Time  0.606 ( 0.600)	Data  0.000 ( 0.015)	Loss 6.7346e+00 (6.9038e+00)	Acc@1   0.00 (  0.35)	Acc@5   0.00 (  1.44)
Epoch: [0][  250/10010]	Time  0.613 ( 0.603)	Data  0.000 ( 0.014)	Loss 6.6537e+00 (6.8605e+00)	Acc@1   0.00 (  0.39)	Acc@5   2.34 (  1.59)
Epoch: [0][  300/10010]	Time  0.625 ( 0.605)	Data  0.000 ( 0.013)	Loss 6.5863e+00 (6.8245e+00)	Acc@1   0.00 (  0.40)	Acc@5   2.34 (  1.72)
Epoch: [0][  350/10010]	Tim

Epoch: [0][ 2950/10010]	Time  0.635 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.5319e+00 (5.6128e+00)	Acc@1   6.25 (  4.86)	Acc@5  28.12 ( 14.21)
Epoch: [0][ 3000/10010]	Time  0.636 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.6137e+00 (5.5974e+00)	Acc@1  12.50 (  4.96)	Acc@5  32.03 ( 14.45)
Epoch: [0][ 3050/10010]	Time  0.631 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.5966e+00 (5.5817e+00)	Acc@1  16.41 (  5.08)	Acc@5  32.81 ( 14.71)
Epoch: [0][ 3100/10010]	Time  0.632 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.5557e+00 (5.5666e+00)	Acc@1  11.72 (  5.19)	Acc@5  28.91 ( 14.94)
Epoch: [0][ 3150/10010]	Time  0.626 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.5847e+00 (5.5517e+00)	Acc@1   8.59 (  5.29)	Acc@5  25.78 ( 15.16)
Epoch: [0][ 3200/10010]	Time  0.635 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.5018e+00 (5.5366e+00)	Acc@1  18.75 (  5.41)	Acc@5  34.38 ( 15.39)
Epoch: [0][ 3250/10010]	Time  0.635 ( 0.632)	Data  0.000 ( 0.009)	Loss 4.3817e+00 (5.5216e+00)	Acc@1  14.84 (  5.53)	Acc@5  33.59 ( 15.64)
Epoch: [0][ 3300/10010]	Tim

Epoch: [0][ 5900/10010]	Time  0.627 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.8504e+00 (4.8899e+00)	Acc@1  21.09 ( 11.11)	Acc@5  43.75 ( 26.42)
Epoch: [0][ 5950/10010]	Time  0.633 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.7890e+00 (4.8801e+00)	Acc@1  25.00 ( 11.22)	Acc@5  43.75 ( 26.60)
Epoch: [0][ 6000/10010]	Time  0.630 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.8917e+00 (4.8699e+00)	Acc@1  23.44 ( 11.32)	Acc@5  43.75 ( 26.78)
Epoch: [0][ 6050/10010]	Time  0.630 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.3733e+00 (4.8599e+00)	Acc@1  28.91 ( 11.43)	Acc@5  53.91 ( 26.96)
Epoch: [0][ 6100/10010]	Time  0.630 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.8204e+00 (4.8502e+00)	Acc@1  21.09 ( 11.53)	Acc@5  45.31 ( 27.14)
Epoch: [0][ 6150/10010]	Time  0.634 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.9303e+00 (4.8409e+00)	Acc@1  21.09 ( 11.63)	Acc@5  42.97 ( 27.30)
Epoch: [0][ 6200/10010]	Time  0.628 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.6640e+00 (4.8314e+00)	Acc@1  27.34 ( 11.73)	Acc@5  46.09 ( 27.47)
Epoch: [0][ 6250/10010]	Tim

Epoch: [0][ 8850/10010]	Time  0.628 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.2083e+00 (4.4166e+00)	Acc@1  30.47 ( 16.43)	Acc@5  53.91 ( 35.04)
Epoch: [0][ 8900/10010]	Time  0.631 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.5698e+00 (4.4102e+00)	Acc@1  23.44 ( 16.51)	Acc@5  53.12 ( 35.16)
Epoch: [0][ 8950/10010]	Time  0.628 ( 0.632)	Data  0.000 ( 0.009)	Loss 3.2349e+00 (4.4040e+00)	Acc@1  34.38 ( 16.59)	Acc@5  58.59 ( 35.27)
Epoch: [0][ 9000/10010]	Time  0.621 ( 0.631)	Data  0.000 ( 0.009)	Loss 3.3172e+00 (4.3974e+00)	Acc@1  28.91 ( 16.66)	Acc@5  57.81 ( 35.40)
Epoch: [0][ 9050/10010]	Time  0.632 ( 0.631)	Data  0.000 ( 0.009)	Loss 3.2078e+00 (4.3908e+00)	Acc@1  30.47 ( 16.75)	Acc@5  60.16 ( 35.52)
Epoch: [0][ 9100/10010]	Time  0.629 ( 0.631)	Data  0.000 ( 0.009)	Loss 3.1004e+00 (4.3846e+00)	Acc@1  39.06 ( 16.82)	Acc@5  57.81 ( 35.63)
Epoch: [0][ 9150/10010]	Time  0.627 ( 0.631)	Data  0.000 ( 0.009)	Loss 2.9085e+00 (4.3783e+00)	Acc@1  38.28 ( 16.90)	Acc@5  60.16 ( 35.75)
Epoch: [0][ 9200/10010]	Tim

Epoch: [1][ 1450/10010]	Time  0.632 ( 0.626)	Data  0.000 ( 0.010)	Loss 3.0917e+00 (3.0496e+00)	Acc@1  32.81 ( 33.66)	Acc@5  53.12 ( 60.35)
Epoch: [1][ 1500/10010]	Time  0.631 ( 0.626)	Data  0.000 ( 0.010)	Loss 3.0171e+00 (3.0479e+00)	Acc@1  37.50 ( 33.67)	Acc@5  57.81 ( 60.39)
Epoch: [1][ 1550/10010]	Time  0.632 ( 0.626)	Data  0.000 ( 0.010)	Loss 2.8785e+00 (3.0459e+00)	Acc@1  38.28 ( 33.70)	Acc@5  60.94 ( 60.43)
Epoch: [1][ 1600/10010]	Time  0.629 ( 0.626)	Data  0.000 ( 0.010)	Loss 2.9102e+00 (3.0442e+00)	Acc@1  36.72 ( 33.76)	Acc@5  62.50 ( 60.46)
Epoch: [1][ 1650/10010]	Time  0.625 ( 0.626)	Data  0.000 ( 0.010)	Loss 2.9943e+00 (3.0411e+00)	Acc@1  31.25 ( 33.81)	Acc@5  66.41 ( 60.52)
Epoch: [1][ 1700/10010]	Time  0.627 ( 0.626)	Data  0.000 ( 0.010)	Loss 2.5545e+00 (3.0388e+00)	Acc@1  40.62 ( 33.85)	Acc@5  69.53 ( 60.56)
Epoch: [1][ 1750/10010]	Time  0.628 ( 0.626)	Data  0.000 ( 0.010)	Loss 3.0450e+00 (3.0370e+00)	Acc@1  33.59 ( 33.91)	Acc@5  53.12 ( 60.59)
Epoch: [1][ 1800/10010]	Tim

...

Epoch: [18][ 5050/10010]	Time  0.622 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.1750e+00 (2.0915e+00)	Acc@1  50.78 ( 50.85)	Acc@5  77.34 ( 76.98)
Epoch: [18][ 5100/10010]	Time  0.621 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.3200e+00 (2.0918e+00)	Acc@1  46.88 ( 50.84)	Acc@5  71.09 ( 76.97)
Epoch: [18][ 5150/10010]	Time  0.619 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.3044e+00 (2.0923e+00)	Acc@1  46.88 ( 50.84)	Acc@5  75.00 ( 76.97)
Epoch: [18][ 5200/10010]	Time  0.617 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.1546e+00 (2.0924e+00)	Acc@1  47.66 ( 50.83)	Acc@5  70.31 ( 76.97)
Epoch: [18][ 5250/10010]	Time  0.619 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.2945e+00 (2.0930e+00)	Acc@1  51.56 ( 50.82)	Acc@5  73.44 ( 76.96)
Epoch: [18][ 5300/10010]	Time  0.621 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.5167e+00 (2.0936e+00)	Acc@1  46.09 ( 50.81)	Acc@5  66.41 ( 76.95)
Epoch: [18][ 5350/10010]	Time  0.614 ( 0.620)	Data  0.000 ( 0.009)	Loss 1.8485e+00 (2.0937e+00)	Acc@1  57.03 ( 50.82)	Acc@5  80.47 ( 76.94)
Epoch: [18][ 5400/10

Epoch: [18][ 8000/10010]	Time  0.618 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.3744e+00 (2.1010e+00)	Acc@1  49.22 ( 50.72)	Acc@5  72.66 ( 76.81)
Epoch: [18][ 8050/10010]	Time  0.618 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.1148e+00 (2.1011e+00)	Acc@1  51.56 ( 50.72)	Acc@5  75.78 ( 76.81)
Epoch: [18][ 8100/10010]	Time  0.614 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.1150e+00 (2.1012e+00)	Acc@1  55.47 ( 50.73)	Acc@5  70.31 ( 76.81)
Epoch: [18][ 8150/10010]	Time  0.620 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.0948e+00 (2.1011e+00)	Acc@1  56.25 ( 50.73)	Acc@5  78.12 ( 76.81)
Epoch: [18][ 8200/10010]	Time  0.615 ( 0.620)	Data  0.000 ( 0.009)	Loss 1.8393e+00 (2.1014e+00)	Acc@1  51.56 ( 50.72)	Acc@5  78.12 ( 76.80)
Epoch: [18][ 8250/10010]	Time  0.617 ( 0.620)	Data  0.000 ( 0.009)	Loss 1.8871e+00 (2.1015e+00)	Acc@1  55.47 ( 50.72)	Acc@5  78.91 ( 76.81)
Epoch: [18][ 8300/10010]	Time  0.618 ( 0.620)	Data  0.000 ( 0.009)	Loss 2.1950e+00 (2.1014e+00)	Acc@1  47.66 ( 50.72)	Acc@5  74.22 ( 76.80)
Epoch: [18][ 8350/10

Epoch: [19][  550/10010]	Time  0.617 ( 0.623)	Data  0.000 ( 0.012)	Loss 2.0195e+00 (2.0376e+00)	Acc@1  50.78 ( 51.79)	Acc@5  82.81 ( 77.88)
Epoch: [19][  600/10010]	Time  0.612 ( 0.623)	Data  0.000 ( 0.011)	Loss 2.1093e+00 (2.0359e+00)	Acc@1  51.56 ( 51.87)	Acc@5  76.56 ( 77.88)
Epoch: [19][  650/10010]	Time  0.616 ( 0.622)	Data  0.000 ( 0.011)	Loss 1.8844e+00 (2.0397e+00)	Acc@1  53.12 ( 51.73)	Acc@5  81.25 ( 77.82)
Epoch: [19][  700/10010]	Time  0.616 ( 0.622)	Data  0.000 ( 0.011)	Loss 2.0375e+00 (2.0415e+00)	Acc@1  47.66 ( 51.74)	Acc@5  80.47 ( 77.80)
Epoch: [19][  750/10010]	Time  0.613 ( 0.622)	Data  0.000 ( 0.011)	Loss 2.0252e+00 (2.0426e+00)	Acc@1  57.03 ( 51.74)	Acc@5  78.91 ( 77.75)
Epoch: [19][  800/10010]	Time  0.611 ( 0.621)	Data  0.000 ( 0.011)	Loss 2.0851e+00 (2.0442e+00)	Acc@1  46.88 ( 51.69)	Acc@5  75.78 ( 77.74)
Epoch: [19][  850/10010]	Time  0.616 ( 0.621)	Data  0.000 ( 0.011)	Loss 2.4364e+00 (2.0487e+00)	Acc@1  44.53 ( 51.63)	Acc@5  67.97 ( 77.69)
Epoch: [19][  900/10

Epoch: [19][ 3500/10010]	Time  0.618 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.0118e+00 (2.0768e+00)	Acc@1  50.00 ( 51.14)	Acc@5  75.00 ( 77.23)
Epoch: [19][ 3550/10010]	Time  0.619 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.0285e+00 (2.0768e+00)	Acc@1  50.78 ( 51.14)	Acc@5  77.34 ( 77.23)
Epoch: [19][ 3600/10010]	Time  0.618 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.1705e+00 (2.0774e+00)	Acc@1  46.88 ( 51.14)	Acc@5  75.78 ( 77.23)
Epoch: [19][ 3650/10010]	Time  0.611 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.1077e+00 (2.0778e+00)	Acc@1  55.47 ( 51.14)	Acc@5  77.34 ( 77.21)
Epoch: [19][ 3700/10010]	Time  0.621 ( 0.618)	Data  0.000 ( 0.009)	Loss 1.9452e+00 (2.0780e+00)	Acc@1  53.12 ( 51.13)	Acc@5  77.34 ( 77.20)
Epoch: [19][ 3750/10010]	Time  0.613 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.1912e+00 (2.0785e+00)	Acc@1  45.31 ( 51.12)	Acc@5  76.56 ( 77.21)
Epoch: [19][ 3800/10010]	Time  0.619 ( 0.618)	Data  0.000 ( 0.009)	Loss 2.2510e+00 (2.0789e+00)	Acc@1  46.09 ( 51.11)	Acc@5  73.44 ( 77.19)
Epoch: [19][ 3850/10

KeyboardInterrupt: 

In [173]:
weights = list(model.parameters())

In [174]:
weights

[Parameter containing:
 tensor([[[[ 3.8362e-02, -1.4293e-01, -2.8124e-01,  ..., -2.0649e-01,
            -4.5692e-02,  4.9017e-02],
           [-8.4938e-02, -3.2173e-01, -3.8361e-01,  ..., -1.5756e-01,
             1.4969e-01,  2.1292e-01],
           [-2.2489e-01, -4.1997e-01, -4.4556e-01,  ...,  3.6018e-02,
             3.3540e-01,  3.8651e-01],
           ...,
           [-3.0152e-01, -3.1979e-01, -2.1017e-01,  ...,  3.9903e-01,
             4.7362e-01,  3.1599e-01],
           [-2.4259e-01, -1.6649e-01, -7.5633e-03,  ...,  4.6768e-01,
             3.9492e-01,  1.5240e-01],
           [-1.5382e-01, -6.3702e-02,  4.9371e-02,  ...,  3.9127e-01,
             2.5934e-01,  4.4165e-02]],
 
          [[-5.4772e-02,  1.4883e-02,  8.1780e-02,  ..., -1.1096e-02,
            -6.5899e-02, -6.2331e-03],
           [-3.5377e-02,  2.3624e-02,  1.9307e-01,  ...,  2.2843e-03,
            -5.9878e-02, -4.6343e-02],
           [ 4.3764e-03,  1.0042e-01,  2.4756e-01,  ..., -1.6909e-02,
            -1.2